In [12]:
# Show imports
import numpy as np
import pandas as pd
import yfinance as yf
import hvplot.pandas
import plotly.graph_objs as go
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
import os
from datetime import datetime, timedelta
import pandas as pd
import mplfinance as mpf
from dotenv import load_dotenv

In [13]:
#pull data from yahoo finance
data =yf.download(tickers='WMT', period='max', interval = '1d', start='2007-01-01')
# Show data
print(data)
# Show data
print(data)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2007-01-03   47.090000   48.299999   47.060001   47.549999   34.164410   
2007-01-04   47.799999   47.990002   47.320000   47.779999   34.329666   
2007-01-05   47.500000   47.799999   47.150002   47.389999   34.049435   
2007-01-08   46.910000   47.310001   46.900002   47.000000   33.769241   
2007-01-09   47.000000   47.669998   47.000000   47.389999   34.049435   
...                ...         ...         ...         ...         ...   
2021-07-26  142.360001  143.860001  141.460007  142.630005  142.630005   
2021-07-27  143.000000  143.000000  141.960007  142.639999  142.639999   
2021-07-28  142.479996  143.250000  141.660004  142.059998  142.059998   
2021-07-29  142.639999  142.850006  142.020004  142.240005  142.240005   
2021-07-30  141.199997  142.960007  141.199

In [14]:
# build dataframe

WMT_df = pd.DataFrame(data)
WMT_df.head(10)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03,47.090000,48.299999,47.060001,47.549999,34.164410,35687300
2007-01-04,47.799999,47.990002,47.320000,47.779999,34.329666,17073000
2007-01-05,47.500000,47.799999,47.150002,47.389999,34.049435,13556900
2007-01-08,46.910000,47.310001,46.900002,47.000000,33.769241,16396400
2007-01-09,47.000000,47.669998,47.000000,47.389999,34.049435,14643200
2007-01-10,47.049999,47.619999,46.509998,47.279999,33.970409,13348100
2007-01-11,47.270000,47.730000,47.209999,47.599998,34.200325,14823200
2007-01-12,47.619999,48.110001,47.209999,47.980000,34.473358,13910800
2007-01-16,47.930000,48.480000,47.790001,48.310001,34.710464,14482400


In [15]:
# build dataframe

WMT_df = pd.DataFrame(dataz

# change df to csv
WMT_csv = WMT_df.to_csv('Data/walmart_csv', index=True)

In [16]:
WMT_df.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03,47.090000,48.299999,47.060001,47.549999,34.164410,35687300
2007-01-04,47.799999,47.990002,47.320000,47.779999,34.329666,17073000
2007-01-05,47.500000,47.799999,47.150002,47.389999,34.049435,13556900
2007-01-08,46.910000,47.310001,46.900002,47.000000,33.769241,16396400
2007-01-09,47.000000,47.669998,47.000000,47.389999,34.049435,14643200
2007-01-10,47.049999,47.619999,46.509998,47.279999,33.970409,13348100
2007-01-11,47.270000,47.730000,47.209999,47.599998,34.200325,14823200
2007-01-12,47.619999,48.110001,47.209999,47.980000,34.473358,13910800
2007-01-16,47.930000,48.480000,47.790001,48.310001,34.710464,14482400


In [17]:
#Show Close Only
WMT_close = WMT_df.drop(columns =['Open','High','Low','Adj Close','Volume'])
WMT_close.head(10)

,Close
Date,
2007-01-03,47.549999
2007-01-04,47.779999
2007-01-05,47.389999
2007-01-08,47.000000
2007-01-09,47.389999
2007-01-10,47.279999
2007-01-11,47.599998
2007-01-12,47.980000
2007-01-16,48.310001


In [18]:
WMT_returns = WMT_close.pct_change().dropna()
WMT_returns

,Close
Date,
2007-01-04,0.004837
2007-01-05,-0.008162
2007-01-08,-0.008230
2007-01-09,0.008298
2007-01-10,-0.002321
...,...
2021-07-26,0.001404
2021-07-27,0.000070
2021-07-28,-0.004066


In [19]:
# Load .env enviroment variables
load_dotenv()
# Access to the News Api
newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Define a function to build news df
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

In [21]:
# Pull all the Walmart Stock headlines data from 2008 
WMT_news_en = newsapi.get_everything(
    q=" WMT AND Walmart AND 2008",
    language="en"
)
# Show the total number of news
print(WMT_news_en["totalResults"])

4
